<a href="https://colab.research.google.com/github/whiskycasablanca/CLIP_Implementation/blob/Test_L2/CLIP_Zeroshot_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone -b Test_L2 --single-branch https://github.com/whiskycasablanca/CLIP_Implementation.git

Cloning into 'CLIP_Implementation'...
remote: Enumerating objects: 13789, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 13789 (delta 2), reused 6 (delta 2), pack-reused 13778 (from 1)
Receiving objects: 100% (13789/13789), 512.80 MiB | 17.57 MiB/s, done.
Resolving deltas: 100% (2/2), done.
Updating files: 100% (13780/13780), done.


In [17]:
#구글 드라이브에 저장된 best_model.pth 불러오기 위함
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
!ls /content/drive/MyDrive/

 10.jpg						 resnet50_original.ipynb
'11 (1).jpg'					'resnet50 sgd.ipynb'
 11.jpg						 resnet50제출용.ipynb
 12.jpg						'resnext101 (1).ipynb'
 1.jpg						 resnext101.ipynb
 20230721_214259.pdf				'resnext50 (1).ipynb'
 2.jpg						 resnext50_sgd,논문그대로.ipynb
'30^F40.docx'					'resnext car16 drop out .ipynb'
'30^F40.gdoc'					'resnext car16 drop out.ipynb'
 35A1814F-12BA-4AB9-A4E8-51261F282A8A.jpeg	 resnext논문그대로전처리적게ipynb.ipynb
 3.jpg						'resnext채널수1 2.ipynb'
 4.jpg						 soccer_Detect_1222.ipynb
 5A7DA6F3-F7DE-47DA-9337-778D6C00B7BF.jpeg	 SoccerDetect.ipynb
 5.jpg						 styletransfer_adam.ipynb
 6.jpg						 style_transfer_adam_random_init.ipynb
 7.jpg						 styletransfer_lbfgs.ipynb
 8.jpg						 Untitled
 9.jpg						 Untitled0.ipynb
 baseline.ipynb					 Untitled10.ipynb
'baseline_updated (1).ipynb'			 Untitled1.ipynb
 best_model0313_10pm.pth			 Untitled2.ipynb
 best_model0313.pth				 Untitled3.ipynb
 best_model.pth					 Untitled4.ipynb
 clip1차결과.ipyn

In [2]:
import sys
sys.path.append('/content/CLIP_Implementation')

In [3]:
import os
os.chdir('/content/CLIP_Implementation')

In [21]:
import os
import glob
import torch
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from transformers import DistilBertTokenizer
from CLIP import CLIPModel
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np

texts = [
    "a photo of a airplane",
    "a photo of a automobile",
    "a photo of a bird",
    "a photo of a cat",
    "a photo of a deer",
    "a photo of a dog",
    "a photo of a frog",
    "a photo of a horse",
    "a photo of a ship",
    "a photo of a truck"
]

class ZeroShotImageDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        """
        Args:
            image_dir (str): 이미지들이 저장된 폴더 경로.
            transform (callable, optional): 이미지에 적용할 변환(transform) 함수.
        """
        self.image_dir = image_dir
        # 지정된 폴더 내 모든 이미지 파일 경로 읽기
        self.image_paths = glob.glob(os.path.join(image_dir, "*.*"))
        self.image_paths = sorted(self.image_paths)
        self.transform = transform
        self.labels = '/content/CLIP_Implementation/labels.txt'
        with open("/content/CLIP_Implementation/labels.txt", "r") as f:
            self.labels = [int(line.strip()) for line in f if line.strip()]

        # 리스트를 PyTorch 텐서로 변환
        self.labels_tensor = torch.tensor(self.labels)

    def __len__(self):
        # 데이터셋에 포함된 이미지 수 반환
        return len(self.image_paths)

    def __getitem__(self, idx):
        # 주어진 인덱스의 이미지 파일 경로 가져오기
        img_path = self.image_paths[idx]
        # 이미지를 RGB 모드로 열기
        image = Image.open(img_path).convert("RGB")
        # transform이 지정되어 있다면 적용
        if self.transform:
            image = self.transform(image)
        return image, self.labels_tensor[idx]

# 예제: Dataset과 DataLoader 사용하기
if __name__ == "__main__":
    # 이미지 전처리 파이프라인 (예: resize, center crop, tensor 변환, 정규화)
    transform = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),                 # PIL 이미지를 Tensor로 변환 (값 범위: [0, 1])
        transforms.Normalize(                  # CLIP의 정규화 mean, std반영
            mean=[0.48145466, 0.4578275, 0.40821073],
            std=[0.26862954, 0.26130258, 0.27577711]
        )
    ])

    # 모델 불러오기
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = CLIPModel(return_logits=True)
    # 저장된 모델 파라미터 로드 (strict=False 옵션으로 누락 키 무시)
    model.load_state_dict(torch.load("/content/drive/MyDrive/best_model0313.pth"), strict=False)
    model.to(device)
    model.eval()

    # 데이터셋 생성
    dataset = ZeroShotImageDataset("/content/CLIP_Implementation/zeroshot_images", transform=transform)
    # DataLoader 생성 (배치 단위로 불러오기)
    dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

    # "a photo of [CLS] 토크나이즈"
    tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
    tokens = tokenizer(texts, padding=True, return_tensors="pt").to(device)

    with torch.no_grad():
        correction = 0.0
        top3_correction = 0.0
        total_size = 0.0
        # 텍스트 임베딩은 고정해 두고 사용
        text_embedding = model.text_encoder(tokens["input_ids"], tokens["attention_mask"])  # shape: (10, proj_dim)
        for image, labels, in tqdm(dataloader, desc='Evaluating', unit='batch'):
            image, labels = image.to(device), labels.to(device)
            # 이미지 임베딩 계산
            image_embedding = model.image_encoder(image)  # shape: (batch, proj_dim)

            # logit 계산: 이미지 임베딩과 텍스트 임베딩의 내적에 temperature 스케일 적용
            logits = image_embedding @ text_embedding.T * torch.exp(model.temperature)
            logits = logits

            correction += (logits.argmax(dim=1) == labels).float().sum().item()
            top3_indices = logits.topk(3, dim=1)[1] # shape: (batch, 3)
            top3_correction += (top3_indices == labels.unsqueeze(1)).any(dim=1).float().sum().item()

            total_size += labels.size(0)

        print(f"\nTop-1 Accuracy : {correction / total_size}")
        print(f"Top-3 Accuracy : {top3_correction / total_size}")

Evaluating: 100%|██████████| 313/313 [01:57<00:00,  2.67batch/s]


Top-1 Accuracy : 0.5405
Top-3 Accuracy : 0.8618
